In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import warnings
from pmdarima import auto_arima
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense

warnings.filterwarnings("ignore")

TCN MODEL

In [3]:
# Load synthetic data from CSV file
df = pd.read_csv("Datasets/Synthetic_data_with_OEE_MME_16_06_24(in).csv")

# Convert 'Date' column to datetime if it's not already in datetime format
df["Date"] = pd.to_datetime(df["Date"])

In [4]:
# Calculate MME (Machine Mechanical Efficiency)
df["MME"] = df["General_Running_Time"] / df["Effective_Running_Time"]

# Calculate OEE (Overall Equipment Effectiveness)
#df["Q"] = 1 - (df["Waste_Packages"] / (df["Waste_Packages"] + df["Good_Packages"]))  # Quality
#df["P"] = 1 - ((df["Nominal_Speed"] - df["Actual_Speed"]) / df["Nominal_Speed"])  # Performance
#df["A"] = 1 - ((df["Ancillary_Time"] + df["Internal_Downtime"] + df["External_Downtime"]) / df["Base_Working_Time"])# Availability
df["OEE"] = df["Q"] * df["P"] * df["A"]  # Q=Quality, P=Performance, A=Availability

In [5]:
# Normalize features
scaler = MinMaxScaler()
df[['Effective_Running_Time', 'General_Running_Time', 'Ancillary_Time', 'Internal_Downtime', 'External_Downtime', 'Waste_Packages', 'Good_Packages', 'Nominal_Speed', 'Actual_Speed']] = scaler.fit_transform(df[['Effective_Running_Time', 'General_Running_Time', 'Ancillary_Time', 'Internal_Downtime', 'External_Downtime', 'Waste_Packages', 'Good_Packages', 'Nominal_Speed', 'Actual_Speed']])

# Prepare data for training
X = df[['Effective_Running_Time', 'General_Running_Time', 'Ancillary_Time', 'Internal_Downtime', 'External_Downtime', 'Waste_Packages', 'Good_Packages', 'Nominal_Speed', 'Actual_Speed']].values
y = df[['MME', 'OEE']].values
dates = df['Date'].values  # Save dates for plotting

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Build the model
model = Sequential([
    Input(shape=(9,1)),  # Define the input shape using Input layer
    Conv1D(filters=64, kernel_size=2, activation='relu'),  # Adjust kernel_size to 2
    MaxPooling1D(pool_size=1),  # Pooling size adjusted for the small input
    Flatten(),
    Dense(50, activation='relu'),
    Dense(2)
])

In [7]:
# Compile the model
model.compile(optimizer = "adam", loss = "mse")

# Reshape input data for Conv1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [8]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_data=(X_test, y_test))

# Predict on test set
y_pred = model.predict(X_test)

# Separate predictions for MME and OEE
y_pred_MME = y_pred[:, 0]  # First column for MME
y_pred_OEE = y_pred[:, 1]  # Second column for OEE

Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.6904 - val_loss: 0.0729
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0911 - val_loss: 0.0502
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0544 - val_loss: 0.0423
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0354 - val_loss: 0.0317
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0291 - val_loss: 0.0213
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0176 - val_loss: 0.0135
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0119 - val_loss: 0.0086
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0073 - val_loss: 0.0052
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0044 - val_loss: 0.0034
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0020 - val_loss: 0.0020
Epoch 12/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014 - val_l

In [9]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calculate RMSE for MME
rmse_MME = mean_squared_error(y_test[:, 0], y_pred_MME, squared=False)
print("Root Mean Squared Error (RMSE) for MME:", rmse_MME)

# Calculate RMSE for OEE
rmse_OEE = mean_squared_error(y_test[:, 1], y_pred_OEE, squared=False)
print("Root Mean Squared Error (RMSE) for OEE:", rmse_OEE)

# Calculate MAE
mae_MME = mean_absolute_error(y_test[:, 0], y_pred_MME)
print("Mean Absolute Error (MAE) for MME:", mae_MME)

# Calculate MAE for OEE
mae_OEE = mean_absolute_error(y_test[:, 1], y_pred_OEE)
print("Mean Absolute Error (MAE) for OEE:", mae_OEE)

# Calculate MAPE for MME
mape_MME = mean_absolute_percentage_error(y_test[:, 0], y_pred_MME)
print("Mean Absolute Percentage Error (MAPE) for MME:", mape_MME, "%")

# Calculate MAPE for OEE
mape_OEE = mean_absolute_percentage_error(y_test[:, 1], y_pred_OEE)
print("Mean Absolute Percentage Error (MAPE) for OEE:", mape_OEE, "%")

Root Mean Squared Error (RMSE) for MME: 0.008044852731061938
Root Mean Squared Error (RMSE) for OEE: 0.006605671026225461
Mean Absolute Error (MAE) for MME: 0.00575620868886667
Mean Absolute Error (MAE) for OEE: 0.005038623082668285
Mean Absolute Percentage Error (MAPE) for MME: 0.44151016142951777 %
Mean Absolute Percentage Error (MAPE) for OEE: 0.810321678498274 %


GRU MODEL

In [12]:
data = pd.read_csv('Datasets/Synthetic_data_with_OEE_MME_16_06_24(in).csv')

In [13]:
# Extract dates before dropping the 'Date' column
dates = data['Date']
data = data.drop(columns=['Date'])

In [14]:
# Split data into features (X) and targets (y)
X = data.drop(columns=['OEE', 'MME'])
y = data[['OEE', 'MME']]

In [16]:
from sklearn.preprocessing import StandardScaler
# Normalize the feature data
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Normalize the target data
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

In [17]:
# Split into training and testing sets
X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(X_scaled, y_scaled, dates, test_size=0.2, random_state=42)

In [18]:
# Function to create sequences for the GRU model
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps + 1):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps - 1])
    return np.array(Xs), np.array(ys)

In [19]:
time_steps = 5
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, time_steps)

In [20]:
# Adjust dates for sequences
def adjust_dates(dates, time_steps):
    return dates[time_steps-1:]

In [21]:
dates_train_seq = adjust_dates(dates_train.values, time_steps)
dates_test_seq = adjust_dates(dates_test.values, time_steps)

In [22]:
# Verify the lengths to ensure they match
print(f'X_train_seq shape: {X_train_seq.shape}, y_train_seq shape: {y_train_seq.shape}')
print(f'X_test_seq shape: {X_test_seq.shape}, y_test_seq shape: {y_test_seq.shape}')
print(f'dates_train_seq length: {len(dates_train_seq)}, dates_test_seq length: {len(dates_test_seq)}')

X_train_seq shape: (433, 5, 13), y_train_seq shape: (433, 2)
X_test_seq shape: (106, 5, 13), y_test_seq shape: (106, 2)
dates_train_seq length: 433, dates_test_seq length: 106


In [24]:
from tensorflow.keras.layers import Input, GRU, Dense

# Define the GRU model with separate outputs for OEE and MME
input_layer = Input(shape=(time_steps, X_train_seq.shape[2]))
gru_layer = GRU(64, return_sequences=True)(input_layer)
gru_layer = GRU(32)(gru_layer)

# Output layers for OEE and MME
output_oee = Dense(1, name='oee_output')(gru_layer)
output_mme = Dense(1, name='mme_output')(gru_layer)

In [26]:
from tensorflow.keras.models import Model

model = Model(inputs=input_layer, outputs=[output_oee, output_mme])

In [27]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [28]:
# Train the model
history = model.fit(
    X_train_seq,
    {'oee_output': y_train_seq[:, 0], 'mme_output': y_train_seq[:, 1]},
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - loss: 1.9163 - val_loss: 1.7963
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.6158 - val_loss: 1.5137
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.1288 - val_loss: 1.2563
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9937 - val_loss: 0.9568
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7079 - val_loss: 0.6820
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4674 - val_loss: 0.4271
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2754 - val_loss: 0.2347
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1273 - val_loss: 0.1343
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0821 - val_loss: 0.0873
Epoch 10/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0515 - val_loss: 0.0735
Epoch 11/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0373 - val_loss: 0.0645
Epoch 12/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0327 - val_

In [29]:
# Evaluate the model
loss = model.evaluate(X_test_seq,
                      {'oee_output': y_test_seq[:, 0], 'mme_output': y_test_seq[:, 1]},
                      verbose=1)
print(f'Test loss: {loss}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0222 
Test loss: 0.02327892556786537


In [30]:
# Predict
y_pred_oee, y_pred_mme = model.predict(X_test_seq)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step


In [31]:
# Inverse transform the predictions
y_pred_oee_inverse = scaler_y.inverse_transform(np.column_stack([y_pred_oee, np.zeros(y_pred_oee.shape)]))[:, 0]
y_pred_mme_inverse = scaler_y.inverse_transform(np.column_stack([np.zeros(y_pred_mme.shape), y_pred_mme]))[:, 1]

y_test_oee_inverse = scaler_y.inverse_transform(np.column_stack([y_test_seq[:, 0], np.zeros(y_test_seq[:, 0].shape)]))[:, 0]
y_test_mme_inverse = scaler_y.inverse_transform(np.column_stack([np.zeros(y_test_seq[:, 1].shape), y_test_seq[:, 1]]))[:, 1]


In [32]:
# Custom function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [33]:
# Calculate evaluation metrics for OEE
rmse_oee = np.sqrt(mean_squared_error(y_test_oee_inverse, y_pred_oee_inverse))
mae_oee = mean_absolute_error(y_test_oee_inverse, y_pred_oee_inverse)
mape_oee = mean_absolute_percentage_error(y_test_oee_inverse, y_pred_oee_inverse)

print(f'OEE - RMSE: {rmse_oee}')
print(f'OEE - MAE: {mae_oee}')
print(f'OEE - MAPE: {mape_oee}')

# Calculate evaluation metrics for MME
rmse_mme = np.sqrt(mean_squared_error(y_test_mme_inverse, y_pred_mme_inverse))
mae_mme = mean_absolute_error(y_test_mme_inverse, y_pred_mme_inverse)
mape_mme = mean_absolute_percentage_error(y_test_mme_inverse, y_pred_mme_inverse)

print(f'MME - RMSE: {rmse_mme}')
print(f'MME - MAE: {mae_mme}')
print(f'MME - MAPE: {mape_mme}')

OEE - RMSE: 0.008635395087504888
OEE - MAE: 0.006766608047461001
OEE - MAPE: 1.0797735424866177
MME - RMSE: 0.024291709626261166
MME - MAE: 0.018512307157206212
MME - MAPE: 1.4343665514553503


In [2]:
import matplotlib.pyplot as plt

# Function to plot the comparison graphs
def plot_comparison(dates, actual, pred_conv1d, pred_gru, title, ylabel):
    plt.figure(figsize=(14, 7))
    plt.plot(dates, actual, label='Actual', color='blue')
    plt.plot(dates, pred_conv1d, label='Conv1D Prediction', color='red')
    plt.plot(dates, pred_gru, label='GRU Prediction', color='green')
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

# Assuming y_pred_MME and y_pred_OEE are predictions from Conv1D model
# y_pred_mme_inverse and y_pred_oee_inverse are predictions from GRU model

# Extract dates for the test set sequences
dates_test_seq = adjust_dates(dates_test.values, time_steps)

# Inverse transform the predictions of the Conv1D model for comparison
y_pred_MME_inverse_conv1d = scaler_y.inverse_transform(np.column_stack([np.zeros(y_pred_MME.shape), y_pred_MME]))[:, 1]
y_pred_OEE_inverse_conv1d = scaler_y.inverse_transform(np.column_stack([y_pred_OEE, np.zeros(y_pred_OEE.shape)]))[:, 0]

# Plot comparison for OEE
plot_comparison(
    dates_test_seq,
    y_test_oee_inverse,
    y_pred_OEE_inverse_conv1d,
    y_pred_oee_inverse,
    title='OEE Comparison: Actual vs Conv1D vs GRU',
    ylabel='OEE'
)

# Plot comparison for MME
plot_comparison(
    dates_test_seq,
    y_test_mme_inverse,
    y_pred_MME_inverse_conv1d,
    y_pred_mme_inverse,
    title='MME Comparison: Actual vs Conv1D vs GRU',
    ylabel='MME'
)


NameError: name 'adjust_dates' is not defined